# Some setup stuff

Here we will be setting up some stuff that will be helpful going forward, such as the location of Bath and the time of the Vernal Equinox of the year 2022.

In [2]:
bath_latitude = 51.37
bath_longitude = -2.36

## Increasing the RA

The Right Ascension in most databases is measured at the vernal equinox, the 20th of March. However, since the Earth moves around the Sun it is not the same all year around. Each day the Earth covers 360/365 degrees of its rotation, and each hour, since it rotates a full 360º it covers 15º. As such, each day the RA increases

$$\Delta RA = \frac{360}{365*15}$$

This cell will end up not being relevant to the calculations, but it is still useful to understand how RA works. This [video](https://www.youtube.com/watch?v=kejolGai7X8&list=PLX2gX-ftPVXU3BfWLJfmXfGD08cIit5CI&index=9) I have found explains it quite well.

## The sidereal time

For these observations we use sidereal time. A sidereal day takes a little less than a solar day, 23h 56 min instead of the usual 24h. This is the time it takes for the Earth to fully rotate around its axis. This is useful because any object crosses the upper meridian when the sidereal time is equal to the object's Right Ascension ([source](https://lco.global/spacebook/sky/sidereal-time/)).

A solar day lasts 24h because it takes that amount of time for the same spot in the Earth to be facing the Sun, since not only does the Earth rotate, but it also moves around the Sun.

Here we are simply computing the sidereal time at Bath right now (whenever _now_ is). This is UTC time, which matches the GMT timezone. At the time of writing this (19th of October), the timezone in Bath is GMT + 1, but that will change on the 30th of October when Daylight saving time is implemented, and we will be back in the GMT/UTC timezone.

In [3]:
# Obtain sidereal time
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy import units as u
import datetime

observing_location = EarthLocation(lat=bath_latitude*u.deg, lon=bath_longitude*u.deg)
observing_time = Time(datetime.datetime.utcnow(), scale='utc', location=observing_location)
LST = observing_time.sidereal_time('mean')

## The sidereal time at the time of observation

Following the same procedure as above, we can compute the sidereal time of the day of observation. I have coded the program to automatically use the day of whenever it's ran on, but it can be changed to any random date. Since we are currently UTC + 1 I have coded the observation to start at 19:15 (20:15 Bath time) and finishing at 1:00 (2:00 Bath time). This has to be changed manually in the 30th, and an hour added to both cases.

Once we have the sidereal times of both the start and the end, we can use it to find which stars are visible.

In [4]:
date_today = datetime.datetime.today().strftime('%Y-%m-%d')
date_tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
times = [f'{date_today}T19:15:00', f'{date_tomorrow}T1:00:00']# In UTC, change on 30th October

t = Time(times, format='isot', scale='utc')

In [5]:
import re

start_end_times = re.split(' ',str(t.sidereal_time('apparent', bath_longitude)))
start_sidereal_time = start_end_times[0].strip('[]')[:-5] # Keeping it to 4 decimals
end_sidereal_time = start_end_times[1].strip('[]')[:-5]

# Formatting it to hh mm ss so it has the same format as the RA

start_sidereal_time = start_sidereal_time.replace('h', ' ').replace('m', ' ')
end_sidereal_time = end_sidereal_time.replace('h', ' ').replace('m', ' ')

## Getting the right ascension of all stars of our query

Here we are importing the dataframe with the stars of the Monson et al paper, which I saved as a pickle in that notebook. We also split the Right Ascension column into hours minutes and seconds to make calculations later on a bit simpler. With such a small dataset, the impact on performance is not huge, and it makes things much easier.

In [6]:
import pandas as pd
simbad_df = pd.read_pickle('Monson_stars_simbad.pkl')

## Getting the declination of all stars of our query

Same thing as before, but now we do it with the declination column. It's split into three: degrees, minutes and seconds.

In [7]:
simbad_df['DEC degrees'] = simbad_df['DEC'].str[0:3].astype(float)
simbad_df['DEC minutes'] = simbad_df['DEC'].str[4:6].astype(float)
simbad_df['DEC seconds'] = simbad_df['DEC'].str[7:].astype(float)

# Finding the hour angle of different stars

In order to actually compute which stars are visible we are using the hour angle, i.e. how far away from the meridian it is at the beginning and end of our observations. In order to calculate the hour angle we are using the following formula:

$$\mathrm{LHA}_{\mathrm{object}} = \mathrm{LST}-\alpha_{\mathrm{object}}$$

where $\mathrm{LHA}_{\mathrm{object}}$ is the Local Hour Angle of the given object, LST is the local sidereal time and $\alpha_{\mathrm{object}}$ is the Right Ascension of the object. We split it into hours, minutes and seconds and then put it back together.

In [8]:
def formatTimedelta(td):
    """
    format a timedelta object to string, in HH:MM:SS format (seconds floored).
    negative timedeltas will be prefixed with a minus, '-'.
    """
    total = td.total_seconds()
    prefix, total = ('-', total*-1) if total < 0 else ('', total)
    h, r = divmod(total, 3600)
    m, s = divmod(r, 60)
    return f"{prefix}{int(h):02d}:{int(m):02d}:{int(s):02d}"

simbad_df['start sidereal time'] = start_sidereal_time
simbad_df['end sidereal time'] = end_sidereal_time

FMT = '%H %M %S.%f'
simbad_df['RA'] = pd.to_datetime(simbad_df['RA'], format = FMT)
simbad_df['start sidereal time'] = pd.to_datetime(simbad_df['start sidereal time'], format = FMT)
simbad_df['end sidereal time'] = pd.to_datetime(simbad_df['end sidereal time'], format = FMT)




simbad_df['Start hour angle'] = ((simbad_df['start sidereal time'] - simbad_df['RA'])).apply(formatTimedelta)
simbad_df['End hour angle'] = ((simbad_df['end sidereal time']-simbad_df['RA'])).apply(formatTimedelta)
# simbad_df['Time observable'] = (simbad_df['Start hour angle'] - simbad_df['End hour angle'])

In [9]:


fmt = '%H:%M:%S'
neg_fmt = '-%H:%M:%S'

midday = datetime.datetime.strptime('1 Jan 1900 12:00:00', '%d %b %Y %H:%M:%S')
midnight = datetime.datetime.strptime('2 Jan 1900 00:00:00', '%d %b %Y %H:%M:%S')

start_ha_series = simbad_df['Start hour angle'].reset_index(drop = True)
end_ha_series = simbad_df['End hour angle'].reset_index(drop = True)
for start_ha, start_ha_index, end_ha, end_ha_index in zip(start_ha_series, start_ha_series.index, end_ha_series, end_ha_series.index):
    try:
        start_dt = datetime.datetime.strptime(start_ha, fmt)
        if start_dt > midday:
            corrected_start_time = midnight-start_dt
            corrected_start_time_string = f"-{corrected_start_time}"
        else:
            corrected_start_time_string = start_dt.strftime('%H:%M:%S')


    except ValueError:
        start_dt = datetime.datetime.strptime(start_ha, neg_fmt)
        if start_dt < midday:
            corrected_start_time_string = f"-{start_dt.strftime('%H:%M:%S')}"
        else:
            corrected_start_time = midnight-start_dt
            corrected_start_time_string = f"-{corrected_start_time}"
    finally:
        start_ha_series.iloc[start_ha_index] = corrected_start_time_string

    try:
        end_dt = datetime.datetime.strptime(end_ha, fmt)
        if end_dt > midday:
            corrected_end_time = midnight-end_dt
            corrected_end_time_string = f"-{corrected_end_time}"
        else:
            corrected_end_time_string = end_dt.strftime('%H:%M:%S')

    except ValueError:
        end_dt = datetime.datetime.strptime(end_ha, neg_fmt)
        if end_dt < midday:
            corrected_end_time_string = f"-{end_dt.strftime('%H:%M:%S')}"
        else:
            corrected_end_time = midnight-end_dt
            corrected_end_time_string = f"-{corrected_end_time}"
    finally:
        end_ha_series.iloc[end_ha_index] = corrected_end_time_string

In [10]:
simbad_df['Start hour angle folded'] = start_ha_series.values
simbad_df['End hour angle folded'] = end_ha_series.values

# Finding the observable stars



## Circumpolar star

A circumpolar star never sets, but it rotates around the celestial north pole. The amount of circumpolar stars depends on latitude. There are, for example, no circumpolar stars in the equator, and all of the stars are circumpolar at the poles. A star will be circumpolar if
$$ \theta + \delta > 90º$$

where $\theta$ is the latitude and $\delta$ is the declination.

In [11]:
circumpolar_stars = simbad_df[simbad_df['DEC degrees'] + bath_latitude > 90]

## Discriminating based on RA and Dec

Aside from the circumpolar stars, we want something that will be close to the Meridian at the time of observation. This means an hour angle of $\pm3$ hrs or less, which amounts to $\pm 45$º in the East to West plane. Each hour it will move 15º, so that something that starts with an HA = 3hrs at 8pm will cross the meridian at 11pm. Using the hour angle allows us to circumvent calculating which stars are not observable in this coordinate, since anything not visible will have an hour angle too large.

In terms of declination, we want something that is visible from Bath. Since we are not at the equator, the stars will not be where its declination says (in the North to South direction), but instead the following correction must be applied:

$$\mathrm{Dec}_{\mathrm{Bath}} = \mathrm{Dec} - \mathrm{lat}$$

This will tell us what angle the star has with respect to the zenith. To avoid light pollution and trees we can limit the observable declination to 30º in each direction, so that we are left with a small circle in the sky with our observable stars.

In [12]:
def compare_ha(hour_angle):
    h, m, s = re.split(':',hour_angle)
    if 3 > int(h) > -3:
        return True
    else:
        return False

def visible_for_calculator(hour_angle):
    h, m, s = re.split(':', hour_angle)
    h = int(h)
    m = int(m)
    s = int(s)
    if 3 > h > 0:
        seconds = 60 - s
        minutes = 60 - m - 1
        hours = 3 - h - 1
        return(f'{hours}:{minutes}:{seconds}')
    elif -3 < h < 0:
        seconds = s
        minutes = m
        hours = 3 - h
        return(f'{hours}:{minutes}:{seconds}')
    elif h == 0:
        if hour_angle[0] == '-':
            seconds = s
            minutes = m
            hours = 3
            return(f'{hours}:{minutes}:{seconds}')
        else:
            seconds = 60 - s
            minutes = 60 - m - 1
            hours = 2
            return(f'{hours}:{minutes}:{seconds}')
    elif -3 < h + 6 <= 3:
        final_hour = h + 6
        foo = final_hour + 3 - 1
        minutes = 60 - m - 1
        seconds = 60 - s
        return(f'{foo}:{minutes}:{seconds}')
    else:
        return("Not observable")



In [13]:
pd.options.mode.chained_assignment = None

ra_observable = simbad_df[simbad_df['Start hour angle folded'].apply(compare_ha)]
ra_observable['Visible for'] = ra_observable['Start hour angle folded'].apply(visible_for_calculator)

dec_observable = ra_observable[(ra_observable['DEC degrees']-bath_latitude < 30) & (ra_observable['DEC degrees']-bath_latitude > -30)]

# dec_observable = pd.concat([dec_observable, circumpolar_stars])
# Here I just get rid of the columns I made for the operations
dec_observable = dec_observable.drop(columns = ['DEC degrees', 'DEC minutes', 'DEC seconds', 'start sidereal time', 'end sidereal time', 'Start hour angle', 'End hour angle'])


In [14]:
dec_observable

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,Distance_method,Distance_bibcode,FLUX_K,FLUX_G,FLUX_R,FLUX_B,SCRIPT_NUMBER_ID,Start hour angle folded,End hour angle folded,Visible for
0,V* UY Cyg,1900-01-01 20:56:28.303000,+30 25 40.327,14,14,0.0102,0.0154,90,A,O,...,paral,2020yCat.1350....0G,9.715,11.017679,NaN,11.37,1,00:21:45,-6:07:43,2:38:15
0,V* XZ Cyg,1900-01-01 19:32:29.305000,+56 23 17.494,14,14,0.0135,0.0147,90,A,O,...,paral,2020yCat.1350....0G,8.884,9.913754,NaN,9.94,1,01:45:44,-7:31:42,1:14:16
0,V* RR Lyr,1900-01-01 19:25:27.912900,+42 47 03.693,14,14,0.0224,0.0242,90,A,O,...,paral,2020yCat.1350....0G,6.648,7.618805,7.6,7.36,1,01:52:46,-7:38:43,1:7:14
0,V* AV Peg,1900-01-01 21:52:02.794500,+22 34 29.376,14,14,0.0144,0.0134,90,A,O,...,paral,2020yCat.1350....0G,9.322,10.531158,NaN,10.07,1,-00:33:48,-5:12:08,3:33:48
0,V* RZ Cep,1900-01-01 22:39:13.176800,+64 51 30.611,14,14,0.0108,0.0099,90,A,O,...,paral,2020yCat.1350....0G,7.968,9.294270,9.3,9.48,1,-01:20:59,-4:24:58,4:20:59


In [15]:
# Drop the duplicates in case there are any
dec_observable = dec_observable.drop_duplicates()

In [16]:
dec_observable['Visible for']

0    2:38:15
0    1:14:16
0     1:7:14
0    3:33:48
0    4:20:59
Name: Visible for, dtype: object

In [22]:
def organizer(string):
    h, m, s = re.split(':', string)
    h = int(h)
    m = int(m)
    s = int(s)
    ts = h * 3600 + m * 60 + s
    return ts

dec_observable['Visible for in seconds'] = dec_observable['Visible for'].apply(organizer)
dec_observable.sort_values(by=['Visible for in seconds'], ascending = False)

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,Distance_bibcode,FLUX_K,FLUX_G,FLUX_R,FLUX_B,SCRIPT_NUMBER_ID,Start hour angle folded,End hour angle folded,Visible for,Visible for in seconds
0,V* RZ Cep,1900-01-01 22:39:13.176800,+64 51 30.611,14,14,0.0108,0.0099,90,A,O,...,2020yCat.1350....0G,7.968,9.294270,9.3,9.48,1,-01:20:59,-4:24:58,4:20:59,15659
0,V* AV Peg,1900-01-01 21:52:02.794500,+22 34 29.376,14,14,0.0144,0.0134,90,A,O,...,2020yCat.1350....0G,9.322,10.531158,NaN,10.07,1,-00:33:48,-5:12:08,3:33:48,12828
0,V* UY Cyg,1900-01-01 20:56:28.303000,+30 25 40.327,14,14,0.0102,0.0154,90,A,O,...,2020yCat.1350....0G,9.715,11.017679,NaN,11.37,1,00:21:45,-6:07:43,2:38:15,9495
0,V* XZ Cyg,1900-01-01 19:32:29.305000,+56 23 17.494,14,14,0.0135,0.0147,90,A,O,...,2020yCat.1350....0G,8.884,9.913754,NaN,9.94,1,01:45:44,-7:31:42,1:14:16,4456
0,V* RR Lyr,1900-01-01 19:25:27.912900,+42 47 03.693,14,14,0.0224,0.0242,90,A,O,...,2020yCat.1350....0G,6.648,7.618805,7.6,7.36,1,01:52:46,-7:38:43,1:7:14,4034
